## 데이터 불러오기 

In [31]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [32]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### 사용할 데이터만 남기기

추천시스템 구성 시 timestamp데이터는 필요없기 때문에 데이터 제거

In [33]:
cols = ['user_id', 'movie_id', 'ratings'] 
ratings = ratings[cols]
ratings.head(10)

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


#### 3점 이상 데이터만 남기기

별점을 시청횟수로 해석한다.
<br/>
유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

In [34]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [35]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [36]:
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


별점의 column name이 counts로 변경된 것을 볼 수 있다.

### movie와 rating데이터 합치기

사용하기 편하게 하기 위해 movie_id 이용 데이터 합침

In [37]:
total_ratings = pd.merge(ratings,movies, on='movie_id')
total_ratings.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


#### 검색하기 쉽게 소문자로 변경

In [38]:
total_ratings['title'] = total_ratings['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
total_ratings.head(10)

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,one flew over the cuckoo's nest (1975),Drama
5,18,1193,4,one flew over the cuckoo's nest (1975),Drama
6,19,1193,5,one flew over the cuckoo's nest (1975),Drama
7,24,1193,5,one flew over the cuckoo's nest (1975),Drama
8,28,1193,3,one flew over the cuckoo's nest (1975),Drama
9,33,1193,5,one flew over the cuckoo's nest (1975),Drama


In [39]:
condition = total_ratings['movie_id']==1
total_ratings.loc[condition]

,user_id,movie_id,counts,title,genre
38620,1,1,5,toy story (1995),Animation|Children's|Comedy
38621,6,1,4,toy story (1995),Animation|Children's|Comedy
38622,8,1,4,toy story (1995),Animation|Children's|Comedy
38623,9,1,5,toy story (1995),Animation|Children's|Comedy
38624,10,1,5,toy story (1995),Animation|Children's|Comedy
...,...,...,...,...,...
40615,6022,1,5,toy story (1995),Animation|Children's|Comedy
40616,6025,1,5,toy story (1995),Animation|Children's|Comedy
40617,6032,1,4,toy story (1995),Animation|Children's|Comedy
40618,6035,1,4,toy story (1995),Animation|Children's|Comedy


## 데이터 탐색하기

추천 모델 만들기 전에 데이터의 기본적인 정보 확인

1. 유저 수, 영화 수, 인기 많은 영화 수
2. 유저들이 몇 개의 영화를 보고 있는지에 대한 통계
3. 유저 count 횟수 중앙값에 대한 통계

#### 유저 수 확인

In [40]:
total_ratings['user_id'].nunique()

6039

#### 영화 수 확인

In [41]:
total_ratings['title'].nunique()

3628

#### 인기많은 영화 수 

In [42]:
movie_count = total_ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

#### 유저별 몇 개의 영화를 보고 있는지에 대한 통계

In [43]:
user_count = total_ratings.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

#### 유저별 count 횟수, 중앙값에 대한 통계

In [44]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = total_ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅

영화취향에 가장 유사한 영화 추천받기
<br/>
추천 시스템의 검증 과정을 위해 영화 데이터셋에 5개 데이터 추가

In [45]:
condition = total_ratings['genre']=='Comedy'
total_ratings.loc[condition]

,user_id,movie_id,counts,title,genre
13595,1,2918,4,ferris bueller's day off (1986),Comedy
13596,10,2918,5,ferris bueller's day off (1986),Comedy
13597,11,2918,5,ferris bueller's day off (1986),Comedy
13598,19,2918,4,ferris bueller's day off (1986),Comedy
13599,22,2918,4,ferris bueller's day off (1986),Comedy
...,...,...,...,...,...
836434,4682,1553,5,late bloomers (1996),Comedy
836456,5153,3463,3,last resort (1994),Comedy
836458,5220,2543,3,six ways to sunday (1997),Comedy
836459,5754,2543,4,six ways to sunday (1997),Comedy


먼저 user와 movie가 indexing 되어 있기 때문에 추가하는 데이터에도 indexing 한 상태로 넣기

In [46]:
my_favorite = ['ferris bueller\'s day off (1986)' , 'late bloomers (1996)' ,'last resort (1994)' ,'six ways to sunday (1997)' ,'smoking/no smoking (1993)']
num_movie = [2918,1553,3463,2543,3530]

my_playlist = pd.DataFrame({'user_id': ['roh']*5,'movie_id': num_movie,'counts':[30]*5, 'title': my_favorite, 'genre':['Comedy']*5})

if not total_ratings.isin({'user_id':['roh']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    total_ratings = total_ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

total_ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,one little indian (1973),Comedy|Drama|Western
836474,5854,3026,4,slaughterhouse (1987),Horror
836475,5854,690,3,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909,4,"five wives, three secretaries and me (1998)",Documentary
836477,5948,1360,5,identification of a woman (identificazione di ...,Drama
0,roh,2918,30,ferris bueller's day off (1986),Comedy
1,roh,1553,30,late bloomers (1996),Comedy
2,roh,3463,30,last resort (1994),Comedy
3,roh,2543,30,six ways to sunday (1997),Comedy
4,roh,3530,30,smoking/no smoking (1993),Comedy


In [47]:
user_unique = total_ratings['user_id'].unique()
title_unique = total_ratings['title'].unique()
len(user_unique), len(title_unique)

(6040, 3628)

In [48]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = total_ratings['user_id'].unique()
title_unique = total_ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['roh'])    
print(title_to_idx['six ways to sunday (1997)'])


6039
3610


In [49]:
temp_user_data = total_ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(total_ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    total_ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

user_id column indexing OK!!


In [50]:
temp_title_data = total_ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(total_ratings):
    print('artist column indexing OK!!')
    total_ratings['title'] = temp_title_data
else:
    print('artist column indexing Fail!!')

artist column indexing OK!!


In [51]:
total_ratings.head()

,user_id,movie_id,counts,title,genre
0,0,1193,5,0,Drama
1,1,1193,5,0,Drama
2,2,1193,4,0,Drama
3,3,1193,4,0,Drama
4,4,1193,5,0,Drama


### CSR  Matrix 구성하기

MF 구성할 Matrix는 데이터 용량이 크기 때문에 CSR(Compressed Sparse Row)Matrix를 사용해야 함
0이 아닌 유효한 데이터로 채워진 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하여 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조이다

In [52]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = total_ratings['user_id'].nunique()
num_title = total_ratings['title'].nunique()

csr_data = csr_matrix((total_ratings.counts, (total_ratings.user_id, total_ratings.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### MF 모델 학습하기

Matrix Factorization(MF)는 2006년 Netflix에서 백만 달러의 상금을 걸고 개최한 자사 추천 시스템의 성능을 10%이상 향상시키는 챌린지를 계기로 알려짐

MF를 사용하면 계산량 측면에서 작아지기 때문에 유리하게 사용할 수 있음
MF모델을 잘 활용하기 위해서는 k-dimension 벡터를 잘 만들어야함

In [53]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

#### AlternatingLeastSquares 클래스 파라미터
1.factors: 유저와 아이템의 벡터를 몇차원으로 할 것인지
2.regularization: 과적합 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3.use_gpu: GPU 사용 여부
4.iterations: epoch와 같은 의미, 데이터를 몇 번 반복해서 학습할 것인지

In [54]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [55]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [56]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 모델 살펴보기

In [57]:
roh, movie_title = user_to_idx['roh'], title_to_idx['six ways to sunday (1997)']
roh_vector, movie_title_vector = als_model.user_factors[roh], als_model.item_factors[movie_title]


In [58]:
roh_vector

array([ 0.637978  ,  1.538698  ,  0.48989117,  1.2720003 , -0.28494057,
        0.46258035, -1.2148305 , -0.8196665 ,  0.80657756, -0.27188414,
       -0.5624769 , -0.2583892 , -0.3535015 ,  1.7671833 ,  1.5985724 ,
       -0.6278137 ,  0.04743255, -2.0732603 , -0.6203377 , -2.2638304 ,
       -0.39848846, -0.02172546, -1.1970067 , -0.897765  ,  0.5531652 ,
       -2.0401187 ,  0.3341664 ,  0.44429353,  0.2247762 , -1.1541004 ,
        0.7462985 ,  0.30965808, -0.7655701 ,  0.03766444, -1.6252129 ,
        0.5450593 ,  1.3501819 ,  2.0552192 ,  1.4108185 , -0.54735446,
       -0.41511577,  1.0552684 ,  0.81093466,  1.1178138 , -0.5644372 ,
        0.5383323 , -1.3659719 , -0.28432584, -0.7611608 ,  0.0904796 ,
       -0.41330108,  2.5562336 ,  0.09529188, -2.206531  , -1.3662543 ,
        0.9191601 ,  1.4747384 , -2.2223039 ,  0.5698864 ,  1.1481421 ,
        1.8242494 , -1.9541315 ,  0.14727193, -0.07644784,  0.69517505,
       -3.1798077 ,  0.987735  , -1.9628063 ,  0.89469993, -0.16

In [59]:
movie_title_vector

array([ 2.16296385e-03,  3.97697184e-03,  1.75926008e-03,  5.20625385e-03,
        8.28828837e-04,  2.10906076e-03, -2.27948325e-03,  8.11821301e-05,
        4.13657026e-03,  9.18210484e-04,  2.54191388e-03,  2.48484430e-04,
       -1.63333584e-03,  4.91930405e-03,  4.10198700e-03, -4.44639554e-05,
        1.18850311e-03, -3.77071812e-03, -1.23027083e-03, -5.22453757e-03,
       -2.86656490e-04,  7.17659132e-04, -1.98246678e-03, -6.00316620e-04,
        2.92027090e-03, -4.22684709e-03,  2.89846654e-03,  7.70666928e-04,
        2.43349234e-03, -1.54429767e-03,  2.76626810e-03,  2.02026591e-03,
       -1.47115008e-03,  1.17072533e-03, -2.25076592e-03,  2.22907239e-03,
        5.29144146e-03,  4.99770418e-03,  3.27173900e-03,  1.20698905e-03,
        4.05017017e-05,  4.73544281e-03,  7.71288120e-04,  2.37052306e-03,
       -6.65286847e-04,  2.82909488e-03, -1.71464577e-03,  2.75079021e-03,
       -6.26941444e-04,  1.68297114e-03,  7.19239237e-04,  6.39738422e-03,
        2.62712711e-03, -

In [60]:
np.dot(roh_vector, movie_title_vector)

0.31911105

### 모델 튜닝하기

In [94]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=600, regularization=0.01, use_gpu=False, iterations=40, dtype=np.float32)

In [95]:
als_model.fit(csr_data_transpose)

  0%|          | 0/40 [00:00<?, ?it/s]

In [96]:
roh, movie_title = user_to_idx['roh'], title_to_idx['six ways to sunday (1997)']
roh_vector, movie_title_vector = als_model.user_factors[roh], als_model.item_factors[movie_title]


In [97]:
np.dot(roh_vector, movie_title_vector)

0.66589403

### 비슷한 영화 추천

In [98]:
favorite_movie = 'one little indian (1973)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3623, 0.99999994),
 (3615, 0.9841705),
 (3589, 0.9840317),
 (3570, 0.98358446),
 (3531, 0.9834634),
 (3035, 0.9832815),
 (3088, 0.98280513),
 (3591, 0.9826865),
 (3499, 0.9825699),
 (3534, 0.9825359),
 (3480, 0.9824147),
 (3549, 0.98233783),
 (3519, 0.98219585),
 (3616, 0.9819024),
 (3156, 0.9816402)]

movie_id, 유사도를 tuple로 반환해줌

In [99]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['one little indian (1973)',
 'chain of fools (2000)',
 'soft toilet seats (1999)',
 'loser (1991)',
 'aiqing wansui (1994)',
 'for ever mozart (1996)',
 'open season (1996)',
 "heaven's burning (1997)",
 'simon sez (1999)',
 'project moon base (1953)',
 'back stage (2000)',
 'make them die slowly (cannibal ferox) (1980)',
 '1-900 (1994)',
 'song of freedom (1936)',
 'running free (2000)']

In [100]:
def get_similar_artist(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [101]:
get_similar_artist('one little indian (1973)')

['one little indian (1973)',
 'chain of fools (2000)',
 'soft toilet seats (1999)',
 'loser (1991)',
 'aiqing wansui (1994)',
 'for ever mozart (1996)',
 'open season (1996)',
 "heaven's burning (1997)",
 'simon sez (1999)',
 'project moon base (1953)']

### 유저에게 영화 추천

In [102]:
user = user_to_idx['roh']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(844, 0.32379565),
 (406, 0.31589237),
 (379, 0.30740076),
 (811, 0.30351073),
 (652, 0.2958883),
 (433, 0.2889169),
 (212, 0.28206095),
 (314, 0.27177447),
 (870, 0.27110216),
 (765, 0.26441363),
 (1896, 0.26366538),
 (1194, 0.25878266),
 (2292, 0.25449526),
 (1147, 0.25253457),
 (271, 0.25237858),
 (312, 0.24820688),
 (999, 0.24546823),
 (326, 0.24234027),
 (1681, 0.24170601),
 (479, 0.24055487)]

In [103]:
[idx_to_title[i[0]] for i in movie_recommended]

['rainmaker, the (1997)',
 'people vs. larry flynt, the (1996)',
 'patriot games (1992)',
 'thomas crown affair, the (1999)',
 'gremlins (1984)',
 'far and away (1992)',
 'bowfinger (1999)',
 'much ado about nothing (1993)',
 'big sleep, the (1946)',
 'ladyhawke (1985)',
 '...and justice for all (1979)',
 'fatal attraction (1987)',
 "astronaut's wife, the (1999)",
 'sixteen candles (1984)',
 'election (1999)',
 'glengarry glen ross (1992)',
 'evil dead ii (dead by dawn) (1987)',
 'top gun (1986)',
 're-animator (1985)',
 'contact (1997)']

In [104]:
title = title_to_idx['one little indian (1973)']
explain = als_model.explain(user, csr_data, itemid=title)

In [105]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('smoking/no smoking (1993)', 0.022692686368903998),
 ('last resort (1994)', 0.02242019928931769),
 ('late bloomers (1996)', 0.021332569441007792),
 ('six ways to sunday (1997)', 0.017784290852950187),
 ("ferris bueller's day off (1986)", -0.00461177649460942)]

#### 회고

Movielens 데이터에는 user와 movie가 각 indexing 되어 있다는 정보를 보고 그대로 사용해도 된다고 판단하여 필요없는 데이터를 제거한 후 CRS Matrix를 구성하였다.
<br/>
하지만 데이터의 크기가 맞지 않아 row index exceed 오류가 생성되어 데이터 구성하는 부분에서 힘들었다...
<br/>
그래서 처음부터 다시 데이터 구성 파악, 필요한 데이터만 저장하여 CRS Matrix를 구성할 수 있었다.
<br/>
또한, AlternatingLeastSquares을 사용하여 학습할 때 factors iterationsf를 다시 설정하여 recommend 시스템을 구성하여 학습한 결과 처음에는 0.3에서 0.6까지 성능을 높일 수 있었다.